In [1]:
import cvxpy as cp
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
from numpy.linalg import inv,eig, pinv, norm
from IPython.display import clear_output
from cvx_code import *

# Functions for estimating and saving results

* `estimate` takes regressors `Phi`, covariance matrices `C0` and `V0` for data generation. It generates `K` data sets with different $\theta_o$ and find an estimate. It uses `hyperest` together with each of the tuning parameters in `lam`, and it also use `ourest` for the tuning-free choice. 
* `run_and_save` just runs `estimate` and saves the results in csv-files.

In [2]:
def estimate(Phi, C0, V0, ourest, hyperest, lam, K=10):
    (n,d) = Phi.shape
    sqrtC0 = sp.linalg.sqrtm(C0)
    sqrtV0 = sp.linalg.sqrtm(V0)
    R = Phi@C0@Phi.T + V0
    invR = inv(R)
    err_all = np.zeros((K, lam.shape[0]))
    err_our = np.zeros((K,))
    err_opt = np.zeros((K,))
    for i in range(0,K):
        theta0 = sqrtC0@np.random.randn(d,1)
        y = Phi@theta0 + sqrtV0@np.random.randn(n,1)
        theta, C, V, our_lam = ourest(y, Phi)
        err_our[i] = norm(theta0.flatten()-theta.flatten())**2
        
        theta = C0@Phi.T@invR@y
        err_opt[i] = norm(theta0.flatten()-theta.flatten())**2
        for j in range(0,lam.shape[0]):
            theta = hyperest(y, Phi, lam[j])
            err = norm(theta0.flatten()-theta)**2
            err_all[i, j] = err
        clear_output(wait=True)
        print(i)
        
    return err_our, err_opt, err_all, our_lam

def run_and_save(name_method, name_case, func, func_hyper, lam, Phi, C0, V0, SNR, dd, K):
    n = Phi.shape[0]
    err_our, err_opt, err_all, our_lam = estimate(Phi, C0, V0, func, func_hyper, lam, K=K)
    file = name_method + str(n) + name_case + str(SNR)
    
    foo = np.vstack((lam, np.mean(err_all, 0)/dd))
    np.savetxt('./images/' + file + '.csv', foo.T, delimiter=",")
    foo2 = np.vstack((our_lam, np.mean(err_our)/dd))
    np.savetxt('./images/' + file + 'o.csv', foo2.T, delimiter=",")
    

In [3]:
K = 10 # Number of MC-iterations

`setup_case` creates a regressor matrix `Phi` and the covariance matrices `C0` and `V0` for data generation.

In [4]:
def setup_case(n=100, SNR=10, d=100, dd=10, outliers=0):
    Phi = np.random.randn(n,d)
    C0 = np.eye(d)
    C0[dd:, dd:] = 0 # Sparse if dd<d
    signal = np.trace(Phi@C0@Phi.T)
    V0 = np.eye(n)*signal/(n*SNR)
    
    out = int(n*outliers) # outliers
    V0[0:out, 0:out] = 25*V0[0:out,0:out]
    return Phi, C0, V0

# L2-L2

In [8]:
nn = [50, 100, 200]
SSNR = [1, 10]

name_method = "l2l2"
lam = np.linspace(0, 2)
func = l2_l2
func_hyper = l2_l2_hyper

# Case 1: 
name_case = "full"
dd = 100
outliers = 0
for n in nn:
    for SNR in SSNR:
        Phi, C0, V0 = setup_case(n=n, dd=dd, SNR=SNR, outliers=outliers)
        run_and_save(name_method, name_case, func, func_hyper, lam, Phi, C0, V0, SNR, dd, K)

# Case 2:
name_case = "sparse"
dd = 10
outliers = 0
for n in nn:
    for SNR in SSNR:
        Phi, C0, V0 = setup_case(n=n, dd=dd, SNR=SNR, outliers=outliers)
        run_and_save(name_method, name_case, func, func_hyper, lam, Phi, C0, V0, SNR, dd, K)


# Case 3: Sparse outliers
name_case = "sparse_outlier"
dd = 10
outliers = 0.2
for n in nn:
    for SNR in SSNR:
        Phi, C0, V0 = setup_case(n=n, dd=dd, SNR=SNR, outliers=outliers)
        run_and_save(name_method, name_case, func, func_hyper, lam, Phi, C0, V0, SNR, dd, K)

49


# L1-L2

In [7]:
nn = [50, 100, 200]
SSNR = [1, 10]

name_method = "l1l2"
lam = np.linspace(0, 2)
func = l1_l2
func_hyper = l1_l2_hyper

# Case 1: 
name_case = "full"
dd = 100
outliers = 0
for n in nn:
    for SNR in SSNR:
        Phi, C0, V0 = setup_case(n=n, dd=dd, SNR=SNR, outliers=outliers)
        run_and_save(name_method, name_case, func, func_hyper, lam, Phi, C0, V0, SNR, dd, K)

# Case 2:
name_case = "sparse"
dd = 10
outliers = 0
for n in nn:
    for SNR in SSNR:
        Phi, C0, V0 = setup_case(n=n, dd=dd, SNR=SNR, outliers=outliers)
        run_and_save(name_method, name_case, func, func_hyper, lam, Phi, C0, V0, SNR, dd, K)


# Case 3: Sparse outliers
name_case = "sparse_outlier"
dd = 10
outliers = 0.2
for n in nn:
    for SNR in SSNR:
        Phi, C0, V0 = setup_case(n=n, dd=dd, SNR=SNR, outliers=outliers)
        run_and_save(name_method, name_case, func, func_hyper, lam, Phi, C0, V0, SNR, dd, K)

49


# L2-WL1

In [6]:

nn = [50, 100, 200]
SSNR = [1, 10]
nn = [200]
SSNR = [10]

name_method = "l2l1"
lam = np.linspace(0, 0.8)
func = l2_l1
func_hyper = l2_l1_hyper

# Case 1: 
name_case = "full"
dd = 100
outliers = 0
for n in nn:
    for SNR in SSNR:
        Phi, C0, V0 = setup_case(n=n, dd=dd, SNR=SNR, outliers=outliers)
        run_and_save(name_method, name_case, func, func_hyper, lam, Phi, C0, V0, SNR, dd, K)

# Case 2:
name_case = "sparse"
dd = 10
outliers = 0
for n in nn:
    for SNR in SSNR:
        Phi, C0, V0 = setup_case(n=n, dd=dd, SNR=SNR, outliers=outliers)
        run_and_save(name_method, name_case, func, func_hyper, lam, Phi, C0, V0, SNR, dd, K)


# Case 3: Sparse outliers
name_case = "sparse_outlier"
dd = 10
outliers = 0.2
for n in nn:
    for SNR in SSNR:
        Phi, C0, V0 = setup_case(n=n, dd=dd, SNR=SNR, outliers=outliers)
        run_and_save(name_method, name_case, func, func_hyper, lam, Phi, C0, V0, SNR, dd, K)

999


# L1-WL1

In [6]:
nn = [50, 100, 200]
SSNR = [1, 10]

name_method = "l1l1"
lam = np.linspace(0, 0.8)
func = l1_l1
func_hyper = l1_l1_hyper

# Case 1: 
name_case = "full"
dd = 100
outliers = 0
for n in nn:
    for SNR in SSNR:
        Phi, C0, V0 = setup_case(n=n, dd=dd, SNR=SNR, outliers=outliers)
        run_and_save(name_method, name_case, func, func_hyper, lam, Phi, C0, V0, SNR, dd, K)

# Case 2:
name_case = "sparse"
dd = 10
outliers = 0
for n in nn:
    for SNR in SSNR:
        Phi, C0, V0 = setup_case(n=n, dd=dd, SNR=SNR, outliers=outliers)
        run_and_save(name_method, name_case, func, func_hyper, lam, Phi, C0, V0, SNR, dd, K)


# Case 3: Sparse outliers
name_case = "sparse_outlier"
dd = 10
outliers = 0.2
for n in nn:
    for SNR in SSNR:
        Phi, C0, V0 = setup_case(n=n, dd=dd, SNR=SNR, outliers=outliers)
        run_and_save(name_method, name_case, func, func_hyper, lam, Phi, C0, V0, SNR, dd, K)

49
